In [1]:
import dspy
import os
from dotenv import load_dotenv

load_dotenv()

turbo = dspy.OpenAI(model='gpt-4o', api_key=os.getenv("OPENAI_API_KEY"), max_tokens=1000)
dspy.settings.configure(lm=turbo)

/Users/ianz/miniconda3/envs/ad/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pydantic import BaseModel, Field
from typing import List

class Step(BaseModel):
    explanation: str
    sql: str

class SQLSample(BaseModel):
    no_reasonable_example: bool = Field(description="Whether it is not possible to provide a reasonable example")
    steps: List[Step] = Field(description="List of steps to execute the SQL query in TiDB Serverless")

class SampleGen(dspy.Signature):
    """You are a technical assistant at TiDB, dedicated to providing users with precise and actionable guidance. 
    Your mission is to ensure that users receive not only accurate answers but also valuable learning opportunities through practical, step-by-step examples. 
    To achieve this, adhere to the following instructions:

    1. Understand the User's question and answer: Carefully review the user's question and answer provided. Ensure you fully grasp the technical context, the core issue, and any relevant background information.
    2. Determine the Feasibility of Providing a Complete Example:
        - Evaluate whether a step-by-step example can be provided to help the user better understand the topic at hand. Consider the technical details involved, 
        and ensure that any example you provide is fully executable without requiring additional adjustments.
        - Ensure the example is comprehensive, and is designed to be directly usable in TiDB Serverless.
    3. Generate and Present a Complete Example:
        - Create a clear, detailed SQLs guide that the user can follow step-by-step. This example should include all necessary SQL commands and should be self-contained without requiring additional adjustments.
        - **Each step should include a single SQL Query (only SQL are allowed)**. he example should be self-contained, requiring no additional adjustments or assumptions from the user. 
        Avoid combining multiple SQL commands within a single step to maintain clarity and prevent confusion.

    By following these instructions, you will help the user not only resolve their current query but also deepen their understanding of the topic through practical application.
    """

    QA_content: str = dspy.InputField(
        desc="The user's query that requires a step-by-step example to be generated."
    )
    sample : SQLSample = dspy.OutputField(
        desc="Step-by-step example to execute the SQL query in TiDB Serverless."
    )


In [3]:
from dspy.functional import TypedPredictor

class SQLGenModule(dspy.Module):
    def __init__(self, dspy_lm: dspy.LM):
        super().__init__()
        self.dspy_lm = dspy_lm
        self.prog = TypedPredictor(SampleGen)

    def forward(self, QA_content: str):
        with dspy.settings.context(lm=self.dspy_lm):
            return self.prog(QA_content=QA_content)

In [4]:
import pandas as pd
samples = pd.read_csv("sql_sample.csv")
samples

,QA_content,no_reasonable_example,steps
0,User question: Does TiDB support FOREIGN KEY?\...,False,"[{""explanation"": ""Create a 'users' table which..."
1,User question: How to implement rolling upgrad...,True,[]


In [5]:
import json
dataset = []
for _, row in samples.iterrows():
    qa = row['QA_content']
    no_reasonable_example = row['no_reasonable_example']
    steps = [Step(explanation=step["explanation"], sql=step["sql"]) for step in json.loads(row['steps'])]

    sample = SQLSample(no_reasonable_example=no_reasonable_example, steps=steps)
    dataset.append(
        dspy.Example(
            QA_content=qa,
            sample=sample
        )
    )

dataset

[Example({'QA_content': 'User question: Does TiDB support FOREIGN KEY?\nAnswer: Yes, TiDB supports FOREIGN KEY constraints starting from version 6.6.0 as an experimental feature. However, there are key considerations:\n    1.    Experimental Status: The feature is experimental and may change or be removed in future releases.\n    2.    Performance Impact: Foreign keys can lead to performance issues, especially with large datasets.\n    3.    Implementation: FOREIGN KEY constraints can be created, modified, and viewed using standard SQL commands (e.g., CREATE TABLE, ALTER TABLE).\n    4.    Upgrade Considerations: If upgrading from a version prior to 6.6.0, foreign keys will not be automatically enforced and must be recreated.\n    5.    Caution Advised: Given the potential performance impacts and experimental status, use foreign keys cautiously in TiDB.\nIn conclusion, while supported, foreign keys in TiDB are still experimental and may have significant performance implications.\n', 's

In [6]:
trainset = [x.with_inputs('QA_content') for x in dataset]
len(trainset)

2

In [7]:
from dspy.teleprompt import BootstrapFewShot
from dspy.functional import TypedPredictor
import traceback

class AssessmentReuslt(BaseModel):
    """The assessment result of the entities and relationships"""

    score: float = Field(
        description="float between 0 and 1 indicating the quality of the sql samples generated, 1 being the best, 0 being the worst"
    )
   
class SQLSampleAssess(dspy.Signature):
    """
    Assess the quality of the SQL samples generated by the model.
    You need to consider the following factors:
    1. Completeness: Ensure that the steps cover all necessary actions to execute the SQL query.
    2. Correctness: Verify that the SQL commands are accurate and error-free.
    3. Clarity: Evaluate the clarity of the explanations and SQL commands provided.
    """

    assessed_samples: SQLSample = dspy.InputField(desc="the assessed sql samples")
    gold_samples: SQLSample = dspy.InputField(desc="the gold sql samples")
    result:AssessmentReuslt = dspy.OutputField(desc="the assessment result")

def assessment_metric(gold, pred, trace=None):
    with dspy.context(lm=turbo):
        try:
            score = TypedPredictor(SQLSampleAssess)(
                assessed_samples=pred.sample,
                gold_samples=gold.sample,
                config={
                    "response_format":{ "type": "json_object" },
                }
            )
        except Exception as e:
            print(f"Error: {e}")
            traceback.print_exc()
            return False
        
    print(f"score: {score.result.score}")

    if trace is None: # if we're doing evaluation or optimization
        #return relationship_score.result.score >= 0.85
        pass

    return score.result.score >= 0.85


teleprompter = BootstrapFewShot(metric=assessment_metric)

# Compile!
compiled_program = teleprompter.compile(SQLGenModule(turbo), trainset=trainset)

100%|██████████| 2/2 [00:00<00:00, 99.83it/s]

score: 0.7
score: 1.0
Bootstrapped 1 full traces after 2 examples in round 0.


In [8]:
compiled_program.save("./sql_sample_gen_program")

[('prog.predictor', Predict(SampleGen(QA_content -> sample
    instructions="You are a technical assistant at TiDB, dedicated to providing users with precise and actionable guidance. \nYour mission is to ensure that users receive not only accurate answers but also valuable learning opportunities through practical, step-by-step examples. \nTo achieve this, adhere to the following instructions:\n\n1. Understand the User's question and answer: Carefully review the user's question and answer provided. Ensure you fully grasp the technical context, the core issue, and any relevant background information.\n2. Determine the Feasibility of Providing a Complete Example:\n    - Evaluate whether a step-by-step example can be provided to help the user better understand the topic at hand. Consider the technical details involved, \n    and ensure that any example you provide is fully executable without requiring additional adjustments.\n    - Ensure the example is comprehensive, and is designed to be

In [ ]:
from app.utils.sql_sample_gen import SQlGenerator

generator = SQlGenerator(turbo)